# Named Entity Recognition (NER)

Named entity recognition (NER) is a branch of natural language processing that focuses on extracting the text of names or other semantically distinct ideas from a larger text and tagging according to its special meaning within a system. NER can be broken into two broad categories: algorithms which use deterministic rules to find names (find all tokens that match the regular expression: `([A-Z][a-z]+)`) and statisical models which make guesses about where an entity begins and ends.

In this class, we'll use `spaCy`'s small English NER (`en_core_web_sm`) model to explore how statistical models can recognize named entities. Then, we'll see some of the practical considerations we must take when working with NER. Our workflow will look like this:
* Reacquaint ourselves with `spaCy`
* Pass a single chapter of Gibbon's *Decline and Fall* into the `spaCy` parser
* Examine the NER results
* Filter out all place names
* Use the Pleiades gazetteer to get coordinates for all valid place names
* Save data to CSV

Finally, you'll have the chance to boil down our process into a function and test it out on other chapters from Gibbon (or other texts). Keep in mind that, for next class, you must have your own version of the data we produce today. That's because, next week, Carolyn Talmadge from the DataLab will be showing us how to turn our CSVs into webmaps and applications. 

## Preparing our data

In [1]:
# redownload spaCy's small model - should see 'requirement already satisfied'
!python -m spacy download en_core_web_sm

     --------------------------------------- 12.8/12.8 MB 11.5 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
import pandas as pd
import spacy
nlp = spacy.load('en_core_web_sm') # good idea to initialize here

In [5]:
!pip install wget

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=80be3c47559b0e262049cbd3cf23371b42570588de978aa96f0f9db647e37d6f
  Stored in directory: c:\users\minni\appdata\local\pip\cache\wheels\04\5f\3e\46cc37c5d698415694d83f607f833f83f0149e49b3af9d0f38
Successfully built wget


In [6]:
# downloading gibbon text from my gh
import wget
import os
if not os.path.isfile('gibbon_text.csv'):
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/gibbon_text.csv')

In [7]:
gibbon_by_chapter = pd.read_csv('gibbon_text.csv').rename(columns={'Unnamed: 0':'chapter'})
gibbon_by_chapter

,chapter,StringText
0,Chapter 1,"\nIn the second century of the Christian era, ..."
1,Chapter 2,\nIt is not alone by the rapidity or extent of...
2,Chapter 3,\nThe obvious definition of a monarchy seems t...
3,Chapter 4,"\nThe mildness of Marcus, which the rigid disc..."
4,Chapter 5,\nThe power of the sword is more sensibly felt...
...,...,...
66,Chapter 67,\nThe respective merits of Rome and Constantin...
67,Chapter 68,\nThe siege of Constantinople by the Turks att...
68,Chapter 69,\nIn the first ages of the decline and fall of...
69,Chapter 70,"\nIn the apprehension of modern times, Petrarc..."


In [8]:
chapter_eight = gibbon_by_chapter['StringText'][9]

## Using `spaCy`'s off-the-shelf NER model

This model was trained on a wide variety of sources, so we can't expect it to be completely accurate. We'll revisit that problem soon. We can train our own NER model and it would do better, but this will take some time to do and requires a lot of set up. If you're interested in training your own model, reach out to me and we can work together on it.

In [9]:
# pass the first chapter into spaCy parser
chapter_eight_doc = nlp(chapter_eight)

In [10]:
for entity in chapter_eight_doc.ents: # can access NER with the .ents attribute
    print(entity.text, entity.label_, sep='\t')

Philip	PERSON
Gallienus	PERSON
twenty years	DATE
Roman	NORP
Philip	GPE
Philip	PERSON
the summer of the year two hundred and forty-nine	DATE
Maesia	GPE
Marinus	PERSON
Philip	PERSON
Maesian	NORP
first	ORDINAL
senate	ORG
Decius	ORG
one	CARDINAL
Philip	PERSON
a very few days	DATE
Philip	GPE
Decius	ORG
Marinus	GPE
Decius	ORG
Maesia	GPE
Italy	GPE
Philip	PERSON
Imperial	ORG
Philip	PERSON
a few days	DATE
Verona	GPE
Rome	GPE
Praetorian	NORP
Decius	ORG
senate	ORG
Augustus	ORG
Philip	PERSON
Italy	GPE
Imperial	ORG
Decius	ORG
a few months	DATE
Danube	PERSON
Goths	PERSON
first	ORDINAL
Roman	NORP
Capitol	FAC
Gaul	GPE
Spain	GPE
Italy	GPE
Western	NORP
Goths	PERSON
the beginning of the sixth century	DATE
Italy	GPE
Goths	PERSON
Ravenna	PERSON
Cassiodorus	ORG
Gothic	NORP
twelve	CARDINAL
Jornandes	ORG
Asiatic	GPE
Scythia	GPE
first	ORDINAL
Goths	PERSON
Scandinavia	LOC
North	LOC
Italy	GPE
Scandinavian	NORP
his days	DATE
Ravenna	PERSON
Goths	PERSON
Baltic	LOC
Ptolemy	GPE
Sweden	GPE
west Gothland	GPE
the middl

In [11]:
# filter by place name
for  entity in chapter_eight_doc.ents:
    if (entity.label_ == 'GPE') or (entity.label_ == 'LOC'):
        print(entity.text)

Philip
Maesia
Philip
Marinus
Maesia
Italy
Verona
Rome
Italy
Gaul
Spain
Italy
Italy
Asiatic
Scythia
Scandinavia
North
Italy
Baltic
Ptolemy
Sweden
west Gothland
North
kindred
Rome
Iceland
Denmark
Sweden
Scandinavia
North
Maeotis
Mithridates
North
Sweden
Baltic
Sweden
Carlscroon
Pomerania
Prussia
Vistula
Thorn
Elbing
Königsberg
Pomerania
Ostrogoths
Heruli
Prussia
South
Amala
Italy
Germany
Prypec
Borysthenes
Poland
Russia
Finland
the Macedonian war
Carpi
the Euxine Sea
Tanais
Germany
Italy
Japan
Ukraine
Borysthenes
Trajan
Maesia
Philip
Maesia
Maesia
Philippopolis
Thrace
Philippopolis
Rome
Carpi
Rome
Philippopolis
North
Maesia
Hostilianus
Rome
Hostilianus
Rome
Aemilianus
Pannonia
Maesia
Italy
Spoleto
Aemilianus
Aemilianus
Rome
North
East
Mars
Gaul
Germany
Aemilianus
Spoleto
Valerian
Valerian
Rome
Valerian
Gallienus
Europe
Pannonia
Germany
the Lower Rhine
Brunswick
Arminius
canton
Lower Germany
Gallienus
Posthumus
Rhine
Spain
Gallienus
Spain
Mauritania
Africa
Upper Saxony
Germany
Usipetes
Al

In [12]:
# putting all of the data into a dictionary
import collections
place_freq = collections.defaultdict(int)
for entity in chapter_eight_doc.ents:
    if (entity.label_ == 'GPE') or (entity.label_ == 'LOC'):
        place_freq[entity.text] += 1 # the utility of defaultdict!
place_freq = dict(place_freq)

In [13]:
# dict -> df
place_freq_df = pd.DataFrame.from_dict(place_freq, orient='index').reset_index().rename(columns={'index':'place_name',0:'frequency'})
place_freq_df

,place_name,frequency
0,Philip,4
1,Maesia,8
2,Marinus,1
3,Italy,13
4,Verona,1
...,...,...
122,Tyrant,1
123,Sicily,2
124,Arabia,1
125,India,1


In [14]:
# downloading the Pleiades data we need (also from my gh)
if not os.path.isfile('places.csv'):
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/places.csv')
if not os.path.isfile('names.csv'):
    wget.download('https://raw.githubusercontent.com/pnadelofficial/FallDHCourseMaterials/main/names.csv')

In [15]:
places = pd.read_csv('places.csv')
places.columns

Index(['created', 'description', 'details', 'provenance', 'title', 'uri', 'id',
       'representative_latitude', 'representative_longitude',
       'bounding_box_wkt'],
      dtype='object')

In [16]:
# let's find 'Rome' in places
places.loc[places['title'] == 'Roma']

,created,description,details,provenance,title,uri,id,representative_latitude,representative_longitude,bounding_box_wkt
21483,2018-06-07T19:48:13Z,The capital of the Roman Republic and Empire.,<p>The Barrington Atlas Directory notes: Roma/...,Barrington Atlas: BAtlas 43 B2 Roma,Roma,https://pleiades.stoa.org/places/423025,423025,41.891775,12.486137,"POLYGON ((12.486137 41.891775, 12.486137 41.89..."


In [17]:
names = pd.read_csv('names.csv')
names.columns

Index(['created', 'description', 'details', 'provenance', 'title', 'uri', 'id',
       'place_id', 'name_type', 'language_tag', 'attested_form',
       'romanized_form_1', 'romanized_form_2', 'romanized_form_3',
       'association_certainty', 'transcription_accuracy',
       'transcription_completeness', 'year_after_which', 'year_before_which'],
      dtype='object')

In [18]:
# let's find 'Rome' in names
names.loc[names['romanized_form_1'] == 'Rome'].place_id

20810    423025
Name: place_id, dtype: int64

In [19]:
def get_pleiades_id(term):
    """
    Iterates through all of the possible names in the names.csv file
    Returns None if no matched names
    """
    name_row = names.loc[names['attested_form'] == term]
    if len(name_row) == 1:
        return int(name_row.place_id.iloc[0])
    else:
        name_row = names.loc[names['romanized_form_1'] == term]
        if len(name_row) == 1:
            return int(name_row.place_id.iloc[0])
        else:
            name_row = names.loc[names['romanized_form_2'] == term]
            if len(name_row) == 1:
                return int(name_row.place_id.iloc[0])
            else:
                name_row = names.loc[names['romanized_form_3'] == term]
                if len(name_row) == 1:
                    return int(name_row.place_id.iloc[0])
                else:
                    return None

The above function looks very complicated, but all it's doing is checking the results of several `loc` and `iloc` calls from our `DataFrame`. You will very rarely see `for` loops when using `pandas`. Instead, you will see programmers taking advantage of the very efficient search methods in `pandas` like `loc`, sometimes called 'broadcasting'. Check out this [Medium post](https://medium.com/@michaeleby1/broadcasting-versus-iteration-6c06539dc1d5) for a further discussion of the benefits of broadcasting over writing loops.

In [20]:
place_freq_df['pleiades_id'] = place_freq_df['place_name'].apply(get_pleiades_id)
place_freq_df

,place_name,frequency,pleiades_id
0,Philip,4,NaN
1,Maesia,8,NaN
2,Marinus,1,NaN
3,Italy,13,NaN
4,Verona,1,383816.0
...,...,...,...
122,Tyrant,1,NaN
123,Sicily,2,462492.0
124,Arabia,1,981506.0
125,India,1,50004.0


Why are there so many `NaN` values? How do we deal with them? How do we want to deal with them?

In [21]:
place_freq_final = place_freq_df.dropna().reset_index(drop=True)
place_freq_final

,place_name,frequency,pleiades_id
0,Verona,1,383816.0
1,Rome,27,423025.0
2,Maeotis,1,825318.0
3,Vistula,1,20639.0
4,Heruli,1,98980.0
5,Tanais,2,648593.0
6,Africa,2,775.0
7,Usipetes,1,109405.0
8,Milan,1,383706.0
9,Macedonia,1,491656.0


Now that we have the Pleiades IDs, we can finally get the coordiantes from `places.csv`. Let's look back at the Roman example. 

In [22]:
rid = place_freq_final.pleiades_id.iloc[0]
places.loc[places['id'] == rid].representative_latitude.iloc[0]

45.44212954633693

In [23]:
# could've just one function here, but not too much trouble to do two
def get_lat(pl_id):
    places_row = places.loc[places['id'] == pl_id]
    if len(places_row) == 1:
        return places_row.representative_latitude.iloc[0]
    
def get_long(pl_id):
    places_row = places.loc[places['id'] == pl_id]
    if len(places_row) == 1:
        return places_row.representative_longitude.iloc[0]

In [24]:
place_freq_final['lat'] = place_freq_final['pleiades_id'].apply(get_lat)
place_freq_final['long'] = place_freq_final['pleiades_id'].apply(get_long)

In [25]:
place_freq_final

,place_name,frequency,pleiades_id,lat,long
0,Verona,1,383816.0,45.442130,10.995736
1,Rome,27,423025.0,41.891775,12.486137
2,Maeotis,1,825318.0,46.072677,36.711669
3,Vistula,1,20639.0,52.216333,21.090355
4,Heruli,1,98980.0,55.500000,12.500000
5,Tanais,2,648593.0,36.717629,32.984265
6,Africa,2,775.0,32.500000,7.500000
7,Usipetes,1,109405.0,50.500000,7.500000
8,Milan,1,383706.0,45.463746,9.188060
9,Macedonia,1,491656.0,41.250000,21.750000


In [26]:
place_freq_final.to_csv('ch8gibbon_places.csv')

## In-class Activity

Now pair up and try to take all of the above steps and turn it into a single function, so that all you need to do is pass in the text of a chapter and you get back a DataFrame with place names and coordinates. To walkthrough the steps again:

* Consider what all of your inputs are
* Use `spaCy` syntax to parse input text
* Filter and count by place name and label
* Use `pandas` to manipulate the data into a useful form

Try out some different chapters. Do you notice anything about the places? Do the types of places or the accuracy of Pleiades get better or worse over the chapters? I encourage you to Google or look up any entities you don't know on Wikipedia.

In [27]:
def find_name_and_coords(gibbon_chapter):
    second_chapter = gibbon_chapter
    second_chapter_doc = nlp(second_chapter)
    
#    for entity in second_chapter_doc.ents: # can access NER with the .ents attribute
#       print(entity.text, entity.label_, sep='\t')

    import collections
    place_freq = collections.defaultdict(int)
    for entity in second_chapter_doc.ents:
        if (entity.label_ == 'GPE') or (entity.label_ == 'LOC'):
            place_freq[entity.text] += 1 # the utility of defaultdict!
    place_freq = dict(place_freq)
    place_freq_df = pd.DataFrame.from_dict(place_freq, orient='index').reset_index().rename(columns={'index':'place_name',0:'frequency'})
    place_freq_df
    
    places = pd.read_csv('places.csv')
    names = pd.read_csv('names.csv')
    #get_pleiades_id(term)

    place_freq_df['pleiades_id'] = place_freq_df['place_name'].apply(get_pleiades_id)
    place_freq_df
    
    place_freq_final = place_freq_df.dropna().reset_index(drop=True)
    place_freq_final
    
    place_freq_final['lat'] = place_freq_final['pleiades_id'].apply(get_lat)
    place_freq_final['long'] = place_freq_final['pleiades_id'].apply(get_long)
    
    print(place_freq_final)
    place_freq_final.to_csv('ch8gibbon_places.csv')

In [40]:
find_name_and_coords()

   place_name  frequency  pleiades_id        lat       long
0        Rome         32     423025.0  41.891775  12.486137
1        Nile          4     727172.0  19.211409  30.567330
2      Athens         11     579885.0  37.972634  23.722746
3      Sparta          1     570685.0  37.077905  22.427298
4       Padua          2     393473.0  45.409561  11.876975
5     Arpinum          1     432700.0  41.648422  13.609876
6     Etruria          1     413122.0  42.758127  11.546721
7      Latium          2     432900.0  41.590543  13.192265
8      Alesia          1     177434.0  47.536622   4.503884
9      Africa          4        775.0  32.500000   7.500000
10  Euphrates          1     912849.0  35.543310  39.606018
11      Capua          1     432754.0  41.086092  14.250207
12     Verona          2     383816.0  45.442130  10.995736
13      Troas          1     550944.0  39.837052  26.336944
14      Milan          1     383706.0  45.463746   9.188060
15     London          1      79574.0  5